# TXT to texts converter
**Task: Convert long document full of input articles to single files as .txt**

In [1]:
import os
import sys
import uuid
import re
import json

def new_uuid():
    return 'uuid:' + str(uuid.uuid4())

src_file = '../../data/promeny_zpravodajstvi/texts.txt'
output_dir = '../../data/promeny_zpravodajstvi/texts'
if not os.path.exists(os.path.dirname(src_file)):
    raise Exception('Source file does not exist')
if not os.path.exists(output_dir):
   os.makedirs(output_dir, exist_ok=True)
else:
    # remove all files in the output directory
    for f in os.listdir(output_dir):
        os.remove(os.path.join(output_dir, f))

Read file + clean lines

In [2]:
with open(src_file, 'r') as f:
    lines = f.readlines()
    lines = [l.strip() for l in lines]

print(f'Loaded {len(lines)} lines from {src_file}')

Loaded 4295 lines from ../../data/promeny_zpravodajstvi/texts.txt


Get article headers

In [3]:
article_header_lines = lines[:230]
article_headers = []
for line in article_header_lines:
    if not line or not re.match(r'^1\.\d+', line):
        continue

    line = re.sub(r'\d+$', '', line.strip()).strip()
    article_headers.append(line)

print(f'Found {len(article_headers)} article headers')

for l in article_headers[:5]:
    print(l)
print('...')
for l in article_headers[-5:]:
    print(l)


Found 101 article headers
1.1. Zelení chtějí hlavu Čunka
1.2. Kamery viděly do soukromí. Soud to potrestal
1.3. Test MF DNES: Přijímačky jsou těžké i pro dospělé
1.4. Sledují každý náš krok
1.5. Vedení ODS: Zvolte Klause opět prezidentem
...
1.97. Omluva
1.98. Čunek: Lhůta pro výpověď z bytu déle než dva roky
1.99. Česká spořitelna prodlužuje provoz na pobočkách
1.100. Vyšší mzdy ve Škodě ohrozí její rozvoj
1.101. Unimex Group loni vydělal téměř miliardu


Clean article lines

In [4]:
article_lines = lines[1012:]

def clean_lines_by_start(lines, start):
    cleaned = []
    for line in lines:
        if not line.startswith(start):
            cleaned.append(line)
    return cleaned

line_count = len(article_lines)

for start in ['Foto', '© 2022', 'Zpět']:
    article_lines = clean_lines_by_start(article_lines, start)

print(f'Remaining {len(article_lines)}/{line_count} article lines after cleaning')

Remaining 3012/3283 article lines after cleaning


Get articles

In [5]:
texts = '\n'.join(lines[1012:]).strip()
print(texts[:100])

# split texts into articles by headers
articles = []
article = []
for line in article_lines:
    # if not line:
    #     continue

    if re.match(r'^1\.\d+', line) and article and line in article_headers:
        if article:
            articles.append(article)
            article = []
    article.append(line)

if article:
    articles.append(article)

articles = [{'header': str(a[0]).strip(), 
             'text': '\n'.join(a).strip()} 
             for a in articles]

articles = [a for a in articles if a['text']]

print(f'Found {len(articles)} articles')
print(articles[0])

1.1. Zelení chtějí hlavu Čunka
TISK , Datum: 02.04.2007 , Zdroj: Mladá fronta DNES , Strana: 1 , Aut
Found 101 articles
{'header': '1.1. Zelení chtějí hlavu Čunka', 'text': '1.1. Zelení chtějí hlavu Čunka\nTISK , Datum: 02.04.2007 , Zdroj: Mladá fronta DNES , Strana: 1 , Autor: PAVLA KUBÁLKOVÁ , Vytištěno: 171 085 , Prodáno: 144 292 , Infotype:\nNepojmenováno , Datum importu: 01.01.0001 00:00 , Čtenost: 612 533 , Rubrika: Titulní  strana , Země: Česko , GRP: 6,81\nVicepremiér svými výroky o Romech rozčilil ODS, zelené, opozici i některé své lidovce a byl označen za xenofoba\n\nPraha - Poslancům a vedení Strany zelených už došla trpělivost s Jiřím Čunkem. Chtějí, aby odešel z vlády. Poslední kapkou\nbyl pro ně páteční výrok vicepremiéra Čunka o tom, že kdo chce být dotován od státu, musí se nejdřív „opálit“. Narážel tím na\nzneužívání sociálních dávek a Romy.\nVčera odpoledne proto předsednictvo Strany zelených rozhodlo, že je nutné o Čunkově dalším setrvání v pozici vicepremiéra a\nmin

Export articles to .txt files in `output_dir`

In [6]:
for article in articles:
    if not article['text'] or not article['header']:
        print(f'Skipping article without text or header')
        print(f'header: {article["header"]}')
        print(f'text: {article["text"]}')
        continue

    file_name = article['header'].replace(' ', '_').replace('/', '_') + '.txt'
    output_file = os.path.join(output_dir, file_name)
    with open(output_file, 'w') as f:
        f.write(article['text'])

print(f'Exported {len(os.listdir(output_dir))} articles to {output_dir}')

Exported 101 articles to ../../data/promeny_zpravodajstvi/texts


Export list of article headers

In [7]:
# export to os.path.dirname(src_file) + '/articles.txt'
article_list_file = os.path.join(os.path.dirname(src_file), 'article_headers.txt')
with open(article_list_file, 'w') as f:
    for article in articles:
        if article['header']:
            f.write(f"{article['header']}\n")

print(f'Exported article list to {article_list_file}')

Exported article list to ../../data/promeny_zpravodajstvi/articles.txt


Export articles in JSON

In [9]:
article_json_file = os.path.join(os.path.dirname(src_file), 'articles.json')
with open(article_json_file, 'w') as f:
    json.dump(articles, f, indent=2, ensure_ascii=False)

# XLS to csv by sheets

In [4]:
import os
import pandas as pd

# Change the working directory to the data directory
os.chdir('../../data/promeny_zpravodajstvi')

excel_file_path = 'ratings.xls'
out_dir = 'ratings_orig_csv'
if not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)
else:
    # remove all files in the output directory
    for f in os.listdir(out_dir):
        os.remove(os.path.join(out_dir, f))


def export_sheet_to_csv(file_path, sheet_name):
    """
    Export a single sheet from an Excel file to a CSV file.
    
    :param file_path: Path to the Excel file.
    :param sheet_name: Name of the sheet to export.
    """
    # Read the specific sheet from the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Define the output CSV file name based on the sheet name
    out_csv_file_name = os.path.join(out_dir, f"{sheet_name}.csv")
    
    # Save the DataFrame to a CSV file
    df.to_csv(out_csv_file_name, index=False)


# Get the list of sheet names
sheet_names = pd.ExcelFile(excel_file_path).sheet_names

# Iterate over each sheet and export it to a CSV file
for sheet_name in sheet_names:
    export_sheet_to_csv(excel_file_path, sheet_name)
    print(sheet_name)

print(f'Exported {len(os.listdir(out_dir))} CSV files to {out_dir}')

kriterium
odstavce
hard_news
politicka_zprava
soukromy_ramec
strategicke
vecne
funkce
vyvazenost
protichudne_hlediska
odborny_zdroj
citace_a_parafraze
pyramida
fakta_a_nazory
negativni_vyzneni
konflikt
nekompetence
skandal
dopad
obtezujici_okolnosti
problemy
rady
personalizace
soukromy_zivot
senzacnost
emoce
morbidnost
reakce
pozadavky
validace
lokalni_dopad
vzdelavani
kontext
dotazy
informace
podpora
Exported 36 CSV files to ratings_orig_csv


# Ratings csv to raters.csv

In [6]:
import os
import sys
import uuid
import re
import pandas as pd
import os

src_dir = '../../data/promeny_zpravodajstvi/ratings_orig_csv'
output_dir = '../../data/promeny_zpravodajstvi/ratings_output_csv'
if not os.path.exists(os.path.dirname(src_dir)):
    raise Exception('Source file does not exist')
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)
else:
    # remove all files in the output directory
    for f in os.listdir(output_dir):
        os.remove(os.path.join(output_dir, f))

Load csv files and check them

In [10]:
# load csv files
csv_files = [f for f in os.listdir(src_dir) if f.endswith('.csv')]
print(f'Found {len(csv_files)} csv files')

for file in csv_files[:2]:
    src_file = os.path.join(src_dir, file)
    df = pd.read_csv(src_file, header=None)
    print(f'Loaded {len(df)} rows from {src_file}')
    print(df)


Found 36 csv files
Loaded 100 rows from ../../data/promeny_zpravodajstvi/ratings_orig_csv/Validace.csv
      0    1    2           3                   4            5              6  \
0   0.0  0.1  0.2  Unnamed: 3             Average  Pairwise CK  Pairwise CK.1   
1   0.0  0.0  0.0         NaN      pairwise PABAK   cols 1 & 2     cols 2 & 3   
2   0.0  0.0  0.0         NaN  0.9259259333333333     0.962963      0.8888889   
3   0.0  0.0  0.0         NaN                 NaN          NaN            NaN   
4   0.0  0.0  0.0         NaN                 NaN          NaN            NaN   
..  ...  ...  ...         ...                 ...          ...            ...   
95  NaN  NaN  NaN         NaN                 NaN          NaN            NaN   
96  NaN  NaN  NaN         NaN                 NaN          NaN            NaN   
97  NaN  NaN  NaN         NaN                 NaN          NaN            NaN   
98  0.0  0.0  0.0         NaN                 NaN          NaN            NaN   
99  0.